<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9165cd74e0cecfce62635028fbcddf26bf152caaf33fd988a91957d6d0748f0e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-06 09:12:07
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16


-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.45 C
-------------------
Today PnL: 17.22 K (0.12%)
Current PnL: -19.88 L (-13.03%)
CY Booked + Current PnL: -7.23 L (-4.74%)
-------------------
Total profit:  2.18 L
Total loss:  -22.06 L
-------------------
Total Booked + Current PnL: 19.75 L (15.72%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.74%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.86 L (60.03%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.3%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.66,65.0,X-LC,5.77,231504.0,21138.0,7640.0,0.74,10.05,3.30,13.68,37.0,2.77,1.61,28.23,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,69.0,M-LC,10.73,173483.0,32643.0,11849.0,0.69,23.18,6.83,31.59,66.0,2.75,1.21,55.81,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.42,53.0,M-SC,1.76,87613.0,-13164.0,13256.0,0.00,-13.06,15.13,0.09,245.0,-0.99,0.61,15.10,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.68,62.0,X-MC,5.61,150220.0,14087.0,16494.0,-0.49,10.35,10.98,22.47,91.0,0.85,1.05,24.67,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.65,40.0,X-MC,2.61,88200.0,-2478.0,17384.0,0.14,-2.73,19.71,16.44,101.0,-0.14,0.61,16.89,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-39.90,54.0,H-SC,16.97,91000.0,-11500.0,52098.0,5.70,-11.22,57.25,39.61,114.0,-0.22,0.63,11.14,AR,ATH,MISC
6,ASIANPAINT,3465.66,-10.71,65.0,X-LC,4.59,225289.0,-26479.0,79685.0,2.95,-10.52,35.37,21.13,27.0,-0.33,1.57,21.18,X40,ATH,PAINTS
49,MASFIN,398.61,-16.44,58.0,H-SC,8.44,95085.0,-2895.0,24494.0,1.33,-2.95,25.76,22.05,152.0,-0.12,0.66,38.21,XR,ATH,FINANCE
80,UNITDSPR,1644.00,-3.53,77.0,X-LC,4.05,252840.0,18696.0,29936.0,1.32,7.99,11.84,20.77,86.0,0.62,1.76,15.46,X40N,NTT,BREWERIES
32,HONAUT,58357.33,-20.78,58.0,X-SC,0.94,111675.0,-16263.0,63398.0,1.31,-12.71,56.77,36.84,143.0,-0.26,0.78,14.33,X40N,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.0,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.03,4.00,XY24,NTT,AUTO
64,SFL,1287.0,37.16,56.0,M-SC,4.31,181103.0,-81115.0,150949.0,-2.13,-30.93,83.35,26.63,229.0,-0.54,1.26,12.60,XY24,NTT,MISC
85,WHIRLPOOL,2270.0,-40.29,53.0,M-SC,5.34,123878.0,9080.0,84968.0,-2.03,7.91,68.59,81.92,223.0,0.11,0.86,46.65,XR,NTT,DURABLES
13,BERGEPAINT,680.0,-20.05,44.0,X-MC,1.14,215035.0,-12304.0,60360.0,-1.21,-5.41,28.07,21.14,106.0,-0.20,1.50,21.20,XY24,NTT,PAINTS
69,STARHEALTH,761.0,18.08,53.0,H-SC,14.00,262779.0,-18344.0,151203.0,-1.13,-6.53,57.54,47.26,144.0,-0.12,1.83,40.79,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,50.0,M-LC,4.96,204875.0,1705.0,40422.0,0.50,0.84,19.73,20.74,55.0,0.04,1.43,7.76,XY25,NTT,FINANCE
31,HINDZINC,730.22,25.96,45.0,M-LC,10.48,205320.0,244.0,112331.0,-0.29,0.12,54.71,54.89,52.0,0.00,1.43,24.24,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.42,53.0,M-SC,1.76,87613.0,-13164.0,13256.0,0.00,-13.06,15.13,0.09,245.0,-0.99,0.61,15.10,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.42,49.0,H-SC,2.73,224037.0,-45630.0,81953.0,0.16,-16.92,36.58,13.47,138.0,-0.56,1.56,13.92,XY24,NTT,PAINTS
17,CERA,9475.0,-20.05,63.0,H-SC,1.66,145578.0,-30325.0,72352.0,0.56,-17.24,49.70,23.89,149.0,-0.42,1.01,25.73,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-24.38,43.0,H-MC,6.00,102210.0,-29625.0,71843.0,-0.03,-22.47,70.29,32.03,98.0,-0.41,0.71,14.48,OX40N,NTT,IT
67,SIS,528.0,2016.58,45.0,H-SC,2.74,85425.0,-25607.0,49213.0,1.21,-23.06,57.61,21.26,156.0,-0.52,0.59,15.42,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,69.0,M-LC,10.73,173483.0,32643.0,11849.0,0.69,23.18,6.83,31.59,66.0,2.75,1.21,55.81,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,50.76,60.0,M-MC,9.97,237636.0,12674.0,154986.0,-0.13,5.63,65.22,74.53,192.0,0.08,1.65,39.70,XY24,BTT,STEEL
56,RECLTD,446.00,45.36,50.0,M-LC,4.96,204875.0,1705.0,40422.0,0.50,0.84,19.73,20.74,55.0,0.04,1.43,7.76,XY25,NTT,FINANCE
31,HINDZINC,730.22,25.96,45.0,M-LC,10.48,205320.0,244.0,112331.0,-0.29,0.12,54.71,54.89,52.0,0.00,1.43,24.24,X5K,ATH,METALS
46,KPIGREEN,731.05,11.71,68.0,H-SC,7.55,133106.0,7809.0,51113.0,-0.72,6.23,38.40,47.03,141.0,0.15,0.93,66.66,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,69.0,M-LC,10.73,173483.0,32643.0,11849.0,0.69,23.18,6.83,31.59,66.0,2.75,1.21,55.81,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,11.71,68.0,H-SC,7.55,133106.0,7809.0,51113.0,-0.72,6.23,38.40,47.03,141.0,0.15,0.93,66.66,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-40.29,53.0,M-SC,5.34,123878.0,9080.0,84968.0,-2.03,7.91,68.59,81.92,223.0,0.11,0.86,46.65,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,57.0,H-SC,5.86,130910.0,7574.0,124050.0,0.07,6.14,94.76,106.72,119.0,0.06,0.91,30.95,AR,ATH,MISC
31,HINDZINC,730.22,25.96,45.0,M-LC,10.48,205320.0,244.0,112331.0,-0.29,0.12,54.71,54.89,52.0,0.00,1.43,24.24,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,43.0,M-LC,5.48,148800.0,-2145.0,111600.0,0.87,-1.42,75.00,72.51,53.0,-0.02,1.04,4.52,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.03,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.89,88461.0,-40209.0,85506.0,-0.03,-31.25,96.66,35.20,219.0,-0.47,0.62,1.53,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-22.12,32.0,X-MC,4.73,195514.0,-4833.0,35329.0,-0.02,-2.41,18.07,15.23,178.0,-0.14,1.36,29.06,X40N,NTT,REALTY
23,DMART,5391.45,-17.97,36.0,X-LC,8.33,209120.0,-1287.0,60457.0,0.00,-0.61,28.91,28.12,38.0,-0.02,1.46,22.87,X40N,ATH,FMCG
53,QUESS,986.00,-49.31,36.0,X-SC,37.68,50370.0,-14636.0,165566.0,0.06,-22.51,328.70,232.18,198.0,-0.09,0.35,0.06,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-30.55,39.0,X-MC,0.94,201015.0,195.0,67601.0,0.00,0.10,33.63,33.76,80.0,0.00,1.40,5.30,X40,ATH,FMCG
41,ITC,452.00,-37.92,49.0,X-LC,1.12,200490.0,352.0,20530.0,0.27,0.18,10.24,10.44,4.0,0.02,1.40,5.75,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.25,46.0,H-LC,1.26,155495.0,-30600.0,78074.0,-0.31,-16.44,50.21,25.51,15.0,-0.39,1.08,14.59,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-7.17,41.0,X-MC,3.00,309771.0,-18220.0,124745.0,-0.76,-5.55,40.27,32.48,92.0,-0.15,2.16,5.24,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-3.53,77.0,X-LC,4.05,252840.0,18696.0,29936.0,1.32,7.99,11.84,20.77,86.0,0.62,1.76,15.46,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.31,36.0,X-SC,37.68,50370.0,-14636.0,165566.0,0.06,-22.51,328.70,232.18,198.0,-0.09,0.35,0.06,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.89,88461.0,-40209.0,85506.0,-0.03,-31.25,96.66,35.20,219.0,-0.47,0.62,1.53,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-28.63,37.0,X-MC,7.19,80570.0,-2940.0,22971.0,0.37,-3.52,28.51,23.98,82.0,-0.13,0.56,2.09,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.74,41.0,X-MC,7.34,217590.0,-45775.0,155098.0,-0.68,-17.38,71.28,41.51,84.0,-0.30,1.51,2.11,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,45.0,X-MC,2.44,183560.0,-54391.0,207037.0,0.16,-22.86,112.79,64.15,174.0,-0.26,1.28,2.97,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.31,36.0,X-SC,37.68,50370.0,-14636.0,165566.0,0.06,-22.51,328.70,232.18,198.0,-0.09,0.35,0.06,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.63,37.0,X-MC,7.19,80570.0,-2940.0,22971.0,0.37,-3.52,28.51,23.98,82.0,-0.13,0.56,2.09,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.46,41.0,X-SC,4.31,81216.0,-63944.0,139870.0,-0.28,-44.05,172.22,52.31,136.0,-0.46,0.57,8.71,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,40.0,X-MC,2.61,88200.0,-2478.0,17384.0,0.14,-2.73,19.71,16.44,101.0,-0.14,0.61,16.89,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.89,88461.0,-40209.0,85506.0,-0.03,-31.25,96.66,35.20,219.0,-0.47,0.62,1.53,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.33,45.0,X-LC,9.88,281069.0,-64887.0,131597.0,-0.00,-18.76,46.82,19.28,1.0,-0.49,1.96,3.91,X40,ATH,IT
39,INFY,2275.00,-19.24,51.0,X-LC,5.11,314942.0,2054.0,169628.0,0.73,0.66,53.86,54.87,3.0,0.01,2.19,8.99,X40,BTT,IT
41,ITC,452.00,-37.92,49.0,X-LC,1.12,200490.0,352.0,20530.0,0.27,0.18,10.24,10.44,4.0,0.02,1.40,5.75,X40,NTT,FMCG
83,VBL,671.64,-16.13,55.0,X-LC,6.85,302252.0,-13590.0,126251.0,0.58,-4.30,41.77,35.67,5.0,-0.11,2.10,8.96,X40N,ATH,FMCG
1,ABB,7934.00,-39.57,54.0,H-LC,5.06,252384.0,-9235.0,128438.0,0.27,-3.53,50.89,45.57,7.0,-0.07,1.76,7.39,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,43.0,L-SC,27.85,78317.0,-35232.0,75286.0,0.79,-31.03,96.13,35.27,268.0,-0.47,0.55,91.68,XR,NTT,HOTELS
7,ASIANTILES,137.00,7516.67,71.0,L-SC,10.70,83584.0,-10226.0,86844.0,0.06,-10.90,103.90,81.67,269.0,-0.12,0.58,61.86,XR,NTT,CERAMICS
49,MASFIN,398.61,-16.44,58.0,H-SC,8.44,95085.0,-2895.0,24494.0,1.33,-2.95,25.76,22.05,152.0,-0.12,0.66,38.21,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.29,53.0,M-SC,5.34,123878.0,9080.0,84968.0,-2.03,7.91,68.59,81.92,223.0,0.11,0.86,46.65,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,57.0,H-SC,5.86,130910.0,7574.0,124050.0,0.07,6.14,94.76,106.72,119.0,0.06,0.91,30.95,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-50.82,78.0,H-SC,7.53,281005.0,-5500.0,262824.0,1.00,-1.92,93.53,89.82,134.0,-0.02,1.96,48.67,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,77.41,69.0,H-SC,2.37,154057.0,-28718.0,139791.0,-0.51,-15.71,90.74,60.77,125.0,-0.21,1.07,37.98,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,11.71,68.0,H-SC,7.55,133106.0,7809.0,51113.0,-0.72,6.23,38.40,47.03,141.0,0.15,0.93,66.66,MH,ATH,POWER
62,SAMMAANCAP,326.00,-197.73,71.0,M-SC,33.12,168750.0,19530.0,124656.0,0.69,13.09,73.87,96.62,208.0,0.16,1.17,82.62,XY25,NTT,FINANCE
68,SONACOMS,806.63,-31.93,67.0,M-SC,6.91,84700.0,-16459.0,56461.0,0.04,-16.27,66.66,39.54,202.0,-0.29,0.59,19.44,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.67
1,25,44.18
2,50,75.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.05
LC    33.03
MC    22.94
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.15
X40N     12.47
XY25     11.90
XR        9.92
AR        7.85
OX40N     6.54
X200      1.76
X5K       1.43
SR        1.05
MH        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.44
X-LC    22.34
X-MC    18.65
M-SC    12.63
X-SC     5.51
M-LC     5.11
H-LC     4.60
H-MC     2.33
M-MC     1.65
L-SC     1.47
L-LC     0.98
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-3.40,36.83
IT,12.37,-18.25,81.60
FINANCE,12.23,-7.58,56.35
MISC,7.75,-16.68,75.53
ELECTRICAL,5.78,-8.96,47.59
PAINTS,5.49,-17.17,34.64
BANKS,4.07,-15.41,82.54
INSURANCE,3.84,-0.40,37.84
AUTO,2.75,-45.74,106.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3264328.0,22
XR,1231573.0,13
AR,1158299.0,9
X40,838563.0,12
X40N,677914.0,9
OX40N,544437.0,9
XY25,465174.0,8
SR,271079.0,2
X5K,112331.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3319275.0,24
M-SC,1231050.0,15
X-MC,1195854.0,14
X-LC,1071646.0,13
X-SC,559004.0,6
H-LC,277703.0,3
M-LC,276202.0,4
H-MC,252288.0,2
L-SC,251220.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1160621.0      6
           AR         828843.0      5
           XR         767497.0      7
M-SC       XY24       707235.0      6
X-MC       XY24       571403.0      4
X-LC       X40        463138.0      5
           XY24       312223.0      2
X-SC       X40N       307932.0      4
X-MC       X40        289919.0      6
H-SC       SR         271079.0      2
           OX40N      240122.0      3
X-LC       X40N       216644.0      3
H-LC       AR         206512.0      2
X-MC       XY25       181194.0      2
H-MC       XY24       180445.0      1
X-SC       XY24       165566.0      1
L-SC       XR         162130.0      2
M-MC       XY24       154986.0      1
X-MC       X40N       153338.0      2
M-SC       OX40N      143382.0      4
           XR         132833.0      2
           XY25       124656.0      1
           AR         122944.0      2
M-LC       X5K        112331.0      1
           XR         111600.0      1
L-SC       OX40N       89090.0      1
X-SC       X40         85506.0      1
X-LC       XY25        79641.0      3
H-MC       OX40N       71843.0      1
H-LC       X200        71191.0      1
L-MC       XR          57513.0      1
H-SC       MH          51113.0      1
M-LC       XY25        40422.0      1
L-LC       XY25        39261.0      1
M-LC       XY24        11849.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
